In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Define the Neural Network Model with Dropout to avoid overfitting
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        
        # Define layers: 9 input features, 3 hidden layers, 1 output
        self.fc1 = nn.Linear(9, 64)  # First hidden layer: 9 inputs, 64 hidden units
        self.fc2 = nn.Linear(64, 128)  # Second hidden layer: 64 inputs, 128 hidden units
        self.fc3 = nn.Linear(128, 64)  # Third hidden layer: 128 inputs, 64 hidden units
        self.output = nn.Linear(64, 1)  # Output layer: 64 inputs, 1 output
        
        # Activation function
        self.relu = nn.ReLU()

        # Dropout layers to avoid overfitting
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        # Define forward pass with dropout
        x = self.relu(self.fc1(x))  # First hidden layer + activation
        x = self.dropout(x)  # Dropout after first hidden layer
        x = self.relu(self.fc2(x))  # Second hidden layer + activation
        x = self.dropout(x)  # Dropout after second hidden layer
        x = self.relu(self.fc3(x))  # Third hidden layer + activation
        x = self.output(x)  # Output layer
        return x

def train_model(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for data, target in train_loader:
            data, target = data.float(), target.float()

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)

            # Backward pass and optimization
            optimizer.zero_grad()  # Zero the gradients
            loss.backward()  # Compute gradients
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            optimizer.step()  # Update weights

            running_loss += loss.item()

        # Print the average loss for this epoch
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader)}")

def main():
    # Read CSV data from 'TrainingData.csv'
    data = pd.read_csv('TrainingData.csv')

    # Filter rows to include only rows from 2 to 9964 (indexing 1 to 9963)
    data = data.iloc[1:9964]

    # Extracting labels and features (1st column is label, 4th to 12th columns are features)
    labels = data.iloc[:, 0].values  # Labels (first column)
    features = data.iloc[:, 3:12].values  # Features (columns 4 to 12)

    # Standardize the features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Convert to torch tensors
    features = torch.tensor(features, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32).view(-1, 1)  # Reshape labels to match output

    # Train-Test Split (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Create DataLoader for batches
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # Create a model instance
    model = SimpleNN()

    # Adam optimizer with lower learning rate and weight decay
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

    # Mean Squared Error loss
    criterion = nn.MSELoss()

    # Number of training epochs
    epochs = 10

    # Train the model and print loss after each epoch
    train_model(model, train_loader, criterion, optimizer, epochs)

if __name__ == '__main__':
    main()


Epoch [1/10], Loss: 0.21798035031557084
Epoch [2/10], Loss: 0.03722482314705849
Epoch [3/10], Loss: 0.02291861955076456
Epoch [4/10], Loss: 0.017884288005530834
Epoch [5/10], Loss: 0.01427257677540183
Epoch [6/10], Loss: 0.012334348533302546
Epoch [7/10], Loss: 0.010657883655279875
Epoch [8/10], Loss: 0.008920916009694338
Epoch [9/10], Loss: 0.007566155962646008
Epoch [10/10], Loss: 0.006815843207761645
